# 7. Simulations with increasing cell counts

To assess the effect of cell count and the number of bootstrapped samples on the ability to identify significant clusters by scanpro, we increment the `mu` parameter of the function `simulate_cell_counts_2`, which result in increased cell count each time. For each `mu` parameter, we generate 100 datasets randomly and run scanpro_boot on each, and the mean cell count is then calculated.

In [1]:
import pandas as pd
import numpy as np
import time
import multiprocessing as mp

from scanpro import scanpro
from scanpro.utils import estimate_params_from_counts, simulate_cell_counts_2, convert_counts_to_df

In [30]:
OUT_PATH = 'results/benchmark'

In [3]:
def generate_dataset(p, a, b, n_reps, n_conds=2, n=20, mu=5000, seed=1):

    np.random.seed(seed)
    if b is None:
        b = a * (1-p) / p

    counts = simulate_cell_counts_2(p, n_reps, a, b, n_conds, n=n, mu=mu)

    counts_df = convert_counts_to_df(counts, column_name="cluster")

    return counts_df

In [4]:
def simulate_datasets(p, a, b, mu, n_reps=4, n_sims=100):
    np.random.seed(10)
    
    pool = mp.Pool()
    jobs = []
    
    for sim in range(n_sims):

        # Run Scanpro or scanpro bootstrapping on 100 datasets
        job = pool.apply_async(generate_dataset, (p, a, b, n_reps), dict(n_conds=2, n=20,
                                                                         mu=mu, seed=sim))
        jobs.append(job)

    pool.close()
    monitor_jobs(jobs)

    results = [job.get() for job in jobs]
    pool.join()

    return results

In [5]:
def monitor_jobs(jobs):
    """
    Monitor the status of jobs submitted to a pool.

    Parameters
    ----------
    jobs : list of job objects
        List of job objects, e.g. as returned by pool.map_async().
    """

    if isinstance(jobs, dict):
        jobs = list(jobs.values())

    from tqdm import tqdm_notebook as tqdm
    pbar = tqdm(total=len(jobs))
    
    # Wait for all jobs to finish
    n_ready = sum([job.ready() for job in jobs])
    while n_ready != len(jobs):
        if n_ready != pbar.n:
            pbar.n = n_ready
            pbar.refresh()
        time.sleep(1)
        n_ready = sum([job.ready() for job in jobs])

    pbar.n = n_ready  # update progress bar to 100%
    pbar.refresh()
    pbar.close()

In [6]:
def test_performance(datasets,
                     n_reps,  # number of samples per condition
                     transform,
                     ):
    """Test the performance of bootstrap scanpro on simulated data.
    :param list datasets: List of datasets as pandas dataframes to run scanpro on
    :param int n_reps: Number of replicates the bootstrap is going to generate
    :param str transform: method of transformation (logit or arcsin)
    :return pandas.DataFrame all_run_results: A dataframe with results from all runs.
    """
    import warnings
    warnings.filterwarnings('ignore')

    jobs = []

    for mu in datasets.keys():
        pool = mp.Pool()
        for dataset in datasets[mu]:

            # Run Scanpro or scanpro bootstrapping on 100 datasets
            job = pool.apply_async(scanpro.scanpro, (dataset,), dict(clusters_col="cluster", conds_col="group", 
                                                                     samples_col=None, n_reps=n_reps,
                                                                     transform=transform, verbosity=0))
            jobs.append(job)

        pool.close()
        monitor_jobs(jobs)

        results = [job.get() for job in jobs]
        pool.join()

    # Collect result
    all_run_results = [result.results for result in results]

    return all_run_results

In [7]:
#### HELPER STATISTCS FUNCTIONS ####

def compare_p_values(p_values, per_cluster=False):
    """ Compare p-values of simulation with differences with expected output"""
    out = []
    for x in p_values:
        if per_cluster:
            # save hit rate per cluster
            hitrate = [x[0]<.05, x[1]>=.05, x[2]>=.05, x[3]<.05, x[4]>=.05, x[5]>=.05, x[6]<0.05]
        else:
            # save hit rate per run
            hitrate = x[0]<0.05 and x[1]>=0.05 and x[2]>=0.05 and x[3]<0.05 and x[4]>=0.05 and x[5]>=0.05 and x[6]<0.05
        out.append(hitrate)

    return out


def get_means(data):
    """Get mean cell count for each mu"""
    return [np.mean([x.shape[0] for x in data[mu]]) for mu in data.keys()]


def calc_hitrates(results, compare=None):
    out = {mu: {method: None for method in list(results[mu].keys())} for mu in results.keys()}
    for mu in results.keys():
        for method in results[mu]:
            p_values = [results[mu][method].iloc[i:i+7, -1] for i in np.arange(0, len(results[mu][method]+1), 7)]

            if compare is None:
                raise ValueError("Please provide a compare function!")
                
            out[mu][method] = np.mean(compare(p_values)) * 100
    
    out = pd.DataFrame(out)
    out = out.reset_index().melt(id_vars='index')
    out.rename(columns={'index': 'method', 'variable': 'mu', 'value': 'hit_rate'}, inplace=True)

    return out


def get_stats(p_values, mean_counts, n_sims):
    """ Calculate sensitivty, specificty and false positive rate """
    from sklearn.metrics import roc_auc_score
    
    methods = list(p_values[mean_counts[0]].keys())
    results = {count: {method: [] for method in methods} for count in mean_counts}
    df_list = []
    # True is significant cell type in original data
    y_test = np.tile([True, False, False, True, False, False, True], n_sims)  
    for i, count in enumerate(mean_counts):
        for method in methods:
            test_df = pd.DataFrame(p_values[count][method])
            test_stat = np.zeros((2,2))
            test_stat[0][0] = sum([sum(test_df.iloc[:,i] < 0.05) for i in [0,3,6]])  # tp
            test_stat[0][1] = sum([sum(test_df.iloc[:,i] < 0.05) for i in [1,2,4,5]])  # fp
            test_stat[1][0] = sum([sum(test_df.iloc[:,i] > 0.05) for i in [0,3,6]])  # fn
            test_stat[1][1] = sum([sum(test_df.iloc[:,i] > 0.05) for i in [1,2,4,5]])  # tn
            
            sens = test_stat[0][0] / test_stat.sum(axis=0)[0]  # true positive rate
            specif = test_stat[1][1] / test_stat.sum(axis=0)[1]  # specificity
            fpr = test_stat[0][1] / test_stat.sum(axis=0)[1]  # false positive rate
            
            results[count][method].append(sens)
            results[count][method].append(specif)
            results[count][method].append(fpr)
            
            # calculate auroc
            y_predicted = np.array(p_values[count][method]).flatten()
            y_predicted = y_predicted < 0.05
        
            auroc = roc_auc_score(y_test, y_predicted)
            results[count][method].append(auroc)
        
        tmp = pd.DataFrame(results[count], index=['sensitivity', 'specificity', 'fpr', 'auroc'])
        tmp = tmp.T
        tmp['mean_counts'] = count
        
        df_list.append(tmp)
    
    df_stat = pd.concat(df_list, axis=0, join='outer')
    df_stat.reset_index(inplace=True)
    df_stat.rename({'index': 'method'}, inplace=True, axis=1)
    df_stat['reps'] = df_stat['method'].str.split('_').str[1]
    df_stat['trans'] = df_stat['method'].str.split('_').str[0]
    
    return df_stat

## Data preperation
see https://phipsonlab.github.io/propeller-paper-analysis/SimTrueDiff.html

In [8]:
heart_counts = pd.read_csv('/home/yalayou/pypropeller/Alayoubi_et_al_2023/data/heart_counts.tsv', sep='\t')
heart_counts.drop(['Condition', 'Sex'], inplace=True, axis=1)
heart_counts = heart_counts.set_index('Sample').T
heart_counts.drop('Erythroid', inplace=True)  # remove erythroids

# proportions of each cluster in all samples
true_props = heart_counts.sum(axis=1) / heart_counts.sum(axis=1).sum()  # sum of cells in cluster / sum of all cells
true_props = true_props.to_frame(name="props")

# estimate beta paramters from counts
params = estimate_params_from_counts(heart_counts)  # rows are clusters
a = params[1]
b = params[2]

# Set up true proportions for the two groups
grp1_trueprops = true_props.values.flatten()
grp2_trueprops = true_props.values.flatten()

grp2_trueprops[0] = grp1_trueprops[0]/2
grp2_trueprops[3] = grp2_trueprops[3]*2
grp2_trueprops[6] = grp1_trueprops[6]*3

grp2_trueprops[0] = grp2_trueprops[0] + (1-grp2_trueprops.sum())/2
grp2_trueprops[3] = grp2_trueprops[3] + (1-grp2_trueprops.sum())
 
# calculate beta for both groups
b1 = a*(1-grp1_trueprops)/grp1_trueprops
b2 = a*(1-grp2_trueprops)/grp2_trueprops
b_grps = [b1, b2]

b_grps

[Cardiomyocytes           1.769390
 Endothelial cells      107.760684
 Epicardial cells        47.297371
 Fibroblast              14.138098
 Immune cells            16.442437
 Neurons                143.134155
 Smooth muscle cells    458.574736
 dtype: float64,
 Cardiomyocytes           4.742325
 Endothelial cells      107.760684
 Epicardial cells        47.297371
 Fibroblast               4.666523
 Immune cells            16.442437
 Neurons                143.134155
 Smooth muscle cells    150.405074
 dtype: float64]

## simulate datasets

In [9]:
n_reps = 4  # number of replicates before merge
mu_list = [250, 500, 750, 1000, 1250, 1500, 2000, 2500, 3500, 5000, 6000, 7000, 9000, 10000, 11000, 12500]

datasets = {f"mu_{mu}": simulate_datasets(true_props, a, b_grps, n_reps=n_reps, mu=mu, n_sims=100) for mu in mu_list}

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
# get mean cell counts for each mu
mean_counts = get_means(datasets)
mean_counts

[1957.24,
 3945.37,
 5913.71,
 7876.67,
 9835.37,
 11808.26,
 15735.6,
 19675.16,
 27532.11,
 39343.57,
 47203.26,
 55074.06,
 70849.16,
 78693.82,
 86561.56,
 98387.63]

## Test bootstrapping with different cell counts

### 2 replicates

In [11]:
n_reps = 2

#### logit

In [12]:
scanpro_2_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [13]:
scanpro_2_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 4 replicates

In [14]:
n_reps = 4

#### logit

In [15]:
scanpro_4_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [16]:
scanpro_4_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 8 replicates

In [17]:
n_reps = 8

#### logit

In [18]:
scanpro_8_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [19]:
scanpro_8_reps_arcsin = test_performance(datasets, transform="arcsin",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 10 reps

In [20]:
n_reps = 10

#### logit

In [21]:
scanpro_10_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [22]:
scanpro_10_reps_arcsin = test_performance(datasets, transform="arcsin",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:01<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 14 reps

In [23]:
n_reps = 14

#### logit

In [24]:
scanpro_14_reps_logit = test_performance(datasets, transform="logit",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [25]:
scanpro_14_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### Results

In [26]:
methods = [f'{trans}_{rep}_reps' for rep in [2,4,8,10,14] for trans in ['logit', 'arcsin']]

results = [scanpro_2_reps_logit, scanpro_2_reps_arcsin,
           scanpro_4_reps_logit, scanpro_4_reps_arcsin,
           scanpro_8_reps_logit, scanpro_8_reps_arcsin,
           scanpro_10_reps_logit, scanpro_10_reps_arcsin,
           scanpro_14_reps_logit, scanpro_14_reps_arcsin]

all_results = {mu: {method: None for method in methods} for mu in mu_list}

for i, method in enumerate(methods):
    for j, mu in enumerate(mu_list):
    
        all_results[mu][method] = pd.concat(results[i][j*100:j*100+100])

In [27]:
# get p_values for each method
p_values = {count: {method: None for method in methods} for count in mean_counts}

for method in methods:
    for i, count in enumerate(mean_counts):

        p_values[count][method] = [all_results[mu_list[i]][method].iloc[j:j+7, -1].to_list() for j in np.arange(0, 700, 7)]

#### Hit rate

In [28]:
hit_rates = calc_hitrates(all_results, compare_p_values)
# add mean cell counts
hit_rates['mean_counts'] = np.repeat(mean_counts, len(methods))
# add transofrmation as seperate column
hit_rates['trans'] = hit_rates['method'].str.split('_').str[0]

hit_rates

,method,mu,hit_rate,mean_counts,trans
0,logit_2_reps,250,0.0,1957.24,logit
1,arcsin_2_reps,250,5.0,1957.24,arcsin
2,logit_4_reps,250,0.0,1957.24,logit
3,arcsin_4_reps,250,2.0,1957.24,arcsin
4,logit_8_reps,250,0.0,1957.24,logit
...,...,...,...,...,...
155,arcsin_8_reps,12500,34.0,98387.63,arcsin
156,logit_10_reps,12500,10.0,98387.63,logit
157,arcsin_10_reps,12500,24.0,98387.63,arcsin
158,logit_14_reps,12500,3.0,98387.63,logit


In [31]:
# save
hit_rates.to_csv(f'{OUT_PATH}/benchmark_hitrates.tsv', sep='\t')

#### specificity, sensitivity and fpr

In [32]:
stats = get_stats(p_values, mean_counts, n_sims=100)
stats

,method,sensitivity,specificity,fpr,auroc,mean_counts,reps,trans
0,logit_2_reps,0.346667,0.9900,0.0100,0.668333,1957.24,2,logit
1,arcsin_2_reps,0.663333,0.9125,0.0875,0.787917,1957.24,2,arcsin
2,logit_4_reps,0.266667,1.0000,0.0000,0.633333,1957.24,4,logit
3,arcsin_4_reps,0.553333,0.9625,0.0375,0.757917,1957.24,4,arcsin
4,logit_8_reps,0.060000,1.0000,0.0000,0.530000,1957.24,8,logit
...,...,...,...,...,...,...,...,...
155,arcsin_8_reps,0.860000,0.8300,0.1700,0.845000,98387.63,8,arcsin
156,logit_10_reps,0.530000,0.9825,0.0175,0.756250,98387.63,10,logit
157,arcsin_10_reps,0.723333,0.9250,0.0750,0.824167,98387.63,10,arcsin
158,logit_14_reps,0.306667,0.9950,0.0050,0.650833,98387.63,14,logit


In [33]:
# save
stats.to_csv(f'{OUT_PATH}/benchmark_stats.tsv', sep='\t')

In [34]:
# get best auroc for each mean count
best_auroc = stats.sort_values('auroc', ascending=False).drop_duplicates('mean_counts').sort_values('mean_counts')
best_auroc

,method,sensitivity,specificity,fpr,auroc,mean_counts,reps,trans
1,arcsin_2_reps,0.663333,0.9125,0.0875,0.787917,1957.24,2,arcsin
11,arcsin_2_reps,0.796667,0.8100,0.1900,0.803333,3945.37,2,arcsin
22,logit_4_reps,0.620000,0.9625,0.0375,0.791250,5913.71,4,logit
30,logit_2_reps,0.863333,0.7575,0.2425,0.810417,7876.67,2,logit
43,arcsin_4_reps,0.810000,0.7900,0.2100,0.800000,9835.37,4,arcsin
53,arcsin_4_reps,0.843333,0.7325,0.2675,0.787917,11808.26,4,arcsin
63,arcsin_4_reps,0.900000,0.7000,0.3000,0.800000,15735.60,4,arcsin
75,arcsin_8_reps,0.640000,0.9675,0.0325,0.803750,19675.16,8,arcsin
85,arcsin_8_reps,0.656667,0.9375,0.0625,0.797083,27532.11,8,arcsin
95,arcsin_8_reps,0.716667,0.9250,0.0750,0.820833,39343.57,8,arcsin


In [35]:
# save
best_auroc.to_csv(f'{OUT_PATH}/benchmark_best_auroc.tsv', sep='\t')